# Retrievers

## 导入依赖

In [4]:
# LangChain 0.3 Retrievers 完整教程

import os
from typing import List, Dict, Any, Optional
import asyncio

# 设置环境变量
os.environ["USER_AGENT"] = "LangChain-Tutorial/1.0 (Educational Purpose)"

# 核心导入
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS, Chroma
from langchain_core.documents import Document
from langchain.retrievers import (
    BM25Retriever,
    EnsembleRetriever,
    MultiQueryRetriever,
    ContextualCompressionRetriever,
    SelfQueryRetriever
)
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.retrievers import (
    TFIDFRetriever,
    SVMRetriever
)

print("✅ 所有库导入成功")

✅ 所有库导入成功


## 1. 准备测试数据

In [5]:

## 1. 准备测试数据

def prepare_test_data():
    """准备测试数据"""
    # 创建测试文档
    documents = [
        Document(
            page_content="人工智能是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。",
            metadata={"source": "ai_intro.txt", "category": "technology"}
        ),
        Document(
            page_content="机器学习是人工智能的一个子集，使计算机能够在没有明确编程的情况下学习和改进。",
            metadata={"source": "ml_intro.txt", "category": "technology"}
        ),
        Document(
            page_content="深度学习是机器学习的一个分支，使用多层神经网络来模拟人脑的工作方式。",
            metadata={"source": "dl_intro.txt", "category": "technology"}
        ),
        Document(
            page_content="自然语言处理是人工智能的一个领域，专注于计算机与人类语言之间的交互。",
            metadata={"source": "nlp_intro.txt", "category": "technology"}
        ),
        Document(
            page_content="计算机视觉是人工智能的一个分支，使计算机能够理解和解释视觉信息。",
            metadata={"source": "cv_intro.txt", "category": "technology"}
        ),
        Document(
            page_content="今天天气很好，阳光明媚，适合户外活动和散步。",
            metadata={"source": "weather.txt", "category": "daily"}
        ),
        Document(
            page_content="Python是一种高级编程语言，广泛用于数据科学、机器学习和Web开发。",
            metadata={"source": "python_intro.txt", "category": "programming"}
        ),
        Document(
            page_content="数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。",
            metadata={"source": "ds_intro.txt", "category": "technology"}
        )
    ]

    # 初始化嵌入模型
    embeddings = OllamaEmbeddings(
        base_url="http://localhost:11434",
        model="nomic-embed-text:latest"
    )

    # 创建向量存储
    vectorstore = FAISS.from_documents(documents, embeddings)

    return documents, vectorstore, embeddings


# 准备数据
documents, vectorstore, embeddings = prepare_test_data()
print(f"✅ 准备了 {len(documents)} 个测试文档")

✅ 准备了 8 个测试文档


## 2. 向量存储检索器

In [7]:


## 2. 向量存储检索器

def vector_store_retriever_example():
    """向量存储检索器示例"""
    print("\n" + "=" * 60)
    print("2. 向量存储检索器示例")
    print("=" * 60)

    # 2.1 基础相似性检索
    print("\n2.1 基础相似性检索")
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )

    query = "什么是机器学习"
    results = retriever.invoke(query)
    print(f"查询: {query}")
    print(f"检索到 {len(results)} 个结果:")
    for i, doc in enumerate(results):
        print(f"{i + 1}. {doc.page_content}")
        print(f"   来源: {doc.metadata.get('source', 'unknown')}")

    # 2.2 相似性阈值检索
    print("\n2.2 相似性阈值检索")
    threshold_retriever = vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.5, "k": 5}
    )

    results = threshold_retriever.invoke("深度学习神经网络")
    print(f"阈值检索结果数量: {len(results)}")

    # 2.3 MMR检索（最大边际相关性）
    print("\n2.3 MMR检索")
    mmr_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.5}
    )

    results = mmr_retriever.invoke("人工智能技术")
    print(f"MMR检索结果数量: {len(results)}")

    return retriever


vector_retriever = vector_store_retriever_example()


2. 向量存储检索器示例

2.1 基础相似性检索
查询: 什么是机器学习
检索到 3 个结果:
1. 数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。
   来源: ds_intro.txt
2. 深度学习是机器学习的一个分支，使用多层神经网络来模拟人脑的工作方式。
   来源: dl_intro.txt
3. 机器学习是人工智能的一个子集，使计算机能够在没有明确编程的情况下学习和改进。
   来源: ml_intro.txt

2.2 相似性阈值检索
阈值检索结果数量: 5

2.3 MMR检索
MMR检索结果数量: 3


## 3. BM25检索器

In [12]:


## 3. BM25检索器

def bm25_retriever_example():
    """BM25检索器示例"""
    print("\n" + "=" * 60)
    print("3. BM25检索器示例")
    print("=" * 60)

    # 3.1 基础BM25检索
    print("\n3.1 基础BM25检索")
    bm25_retriever = BM25Retriever.from_documents(documents)
    bm25_retriever.k = 3

    query = "机器学习算法"
    results = bm25_retriever.invoke(query)
    print(f"查询: {query}")
    print(f"BM25检索到 {len(results)} 个结果:")
    for i, doc in enumerate(results):
        print(f"{i + 1}. {doc.page_content[:80]}...")

    # 3.2 自定义BM25参数
    print("\n3.2 自定义BM25参数")
    custom_bm25 = BM25Retriever.from_documents(
        documents,
        k=2,
        # BM25参数可以通过底层库调整
    )

    results = custom_bm25.invoke("人工智能发展")
    print(f"自定义BM25检索结果数量: {len(results)}")

    return bm25_retriever


bm25_retriever = bm25_retriever_example()



3. BM25检索器示例

3.1 基础BM25检索
查询: 机器学习算法
BM25检索到 3 个结果:
1. 数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。...
2. Python是一种高级编程语言，广泛用于数据科学、机器学习和Web开发。...
3. 今天天气很好，阳光明媚，适合户外活动和散步。...

3.2 自定义BM25参数
自定义BM25检索结果数量: 2


## 4. 集成检索器

In [13]:

## 4. 集成检索器

def ensemble_retriever_example():
    """集成检索器示例"""
    print("\n" + "=" * 60)
    print("4. 集成检索器示例")
    print("=" * 60)

    # 4.1 向量+BM25集成
    print("\n4.1 向量+BM25集成检索")
    ensemble_retriever = EnsembleRetriever(
        retrievers=[vector_retriever, bm25_retriever],
        weights=[0.7, 0.3]  # 向量搜索权重70%，BM25权重30%
    )

    query = "深度学习神经网络"
    results = ensemble_retriever.invoke(query)
    print(f"查询: {query}")
    print(f"集成检索到 {len(results)} 个结果:")
    for i, doc in enumerate(results):
        print(f"{i + 1}. {doc.page_content[:80]}...")

    # 4.2 多种检索器集成
    print("\n4.2 多种检索器集成")
    try:
        # 添加TF-IDF检索器
        tfidf_retriever = TFIDFRetriever.from_documents(documents)
        tfidf_retriever.k = 3

        multi_ensemble = EnsembleRetriever(
            retrievers=[vector_retriever, bm25_retriever, tfidf_retriever],
            weights=[0.5, 0.3, 0.2]
        )

        results = multi_ensemble.invoke("计算机视觉技术")
        print(f"多重集成检索结果数量: {len(results)}")

    except Exception as e:
        print(f"多重集成检索器创建失败: {e}")

    return ensemble_retriever


ensemble_retriever = ensemble_retriever_example()



4. 集成检索器示例

4.1 向量+BM25集成检索
查询: 深度学习神经网络
集成检索到 5 个结果:
1. 数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。...
2. 深度学习是机器学习的一个分支，使用多层神经网络来模拟人脑的工作方式。...
3. 机器学习是人工智能的一个子集，使计算机能够在没有明确编程的情况下学习和改进。...
4. Python是一种高级编程语言，广泛用于数据科学、机器学习和Web开发。...
5. 今天天气很好，阳光明媚，适合户外活动和散步。...

4.2 多种检索器集成
多重集成检索器创建失败: Could not import scikit-learn, please install with `pip install scikit-learn`.


## 5. 多查询检索器

In [ ]:

## 5. 多查询检索器

def multi_query_retriever_example():
    """多查询检索器示例"""
    print("\n" + "=" * 60)
    print("5. 多查询检索器示例")
    print("=" * 60)

    try:
        # 初始化LLM
        llm = ChatOllama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=0.1
        )

        # 5.1 基础多查询检索
        print("\n5.1 基础多查询检索")
        multi_query_retriever = MultiQueryRetriever.from_llm(
            retriever=vector_retriever,
            llm=llm
        )

        query = "AI的应用领域"
        results = multi_query_retriever.invoke(query)
        print(f"查询: {query}")
        print(f"多查询检索到 {len(results)} 个结果:")
        for i, doc in enumerate(results):
            print(f"{i + 1}. {doc.page_content[:80]}...")

        # 5.2 自定义查询生成
        print("\n5.2 自定义查询生成")
        from langchain.prompts import PromptTemplate

        custom_prompt = PromptTemplate(
            input_variables=["question"],
            template="""你是一个AI助手。给定一个用户问题，生成3个不同的搜索查询来检索相关信息。

原始问题: {question}

生成3个相关的搜索查询:
1."""
        )

        custom_multi_retriever = MultiQueryRetriever.from_llm(
            retriever=vector_retriever,
            llm=llm,
            prompt=custom_prompt
        )

        results = custom_multi_retriever.invoke("机器学习的发展历史")
        print(f"自定义多查询检索结果数量: {len(results)}")

        return multi_query_retriever

    except Exception as e:
        print(f"多查询检索器创建失败: {e}")
        return None


multi_query_retriever = multi_query_retriever_example()

## 6. 上下文压缩检索器

In [14]:


## 6. 上下文压缩检索器

def contextual_compression_retriever_example():
    """上下文压缩检索器示例"""
    print("\n" + "=" * 60)
    print("6. 上下文压缩检索器示例")
    print("=" * 60)

    try:
        llm = ChatOllama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=0
        )

        # 6.1 LLM链提取器
        print("\n6.1 LLM链提取器")
        compressor = LLMChainExtractor.from_llm(llm)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=vector_retriever
        )

        query = "什么是深度学习"
        compressed_docs = compression_retriever.invoke(query)
        print(f"查询: {query}")
        print(f"压缩后文档数量: {len(compressed_docs)}")
        for i, doc in enumerate(compressed_docs):
            print(f"{i + 1}. {doc.page_content}")

        return compression_retriever

    except Exception as e:
        print(f"上下文压缩检索器创建失败: {e}")
        return None


compression_retriever = contextual_compression_retriever_example()



6. 上下文压缩检索器示例

6.1 LLM链提取器
上下文压缩检索器创建失败: model "qwen2.5:3b" not found, try pulling it first (status code: 404)


## 7. 自查询检索器

In [15]:

## 7. 自查询检索器

def self_query_retriever_example():
    """自查询检索器示例"""
    print("\n" + "=" * 60)
    print("7. 自查询检索器示例")
    print("=" * 60)

    try:
        from langchain.chains.query_constructor.base import AttributeInfo
        from langchain.retrievers.self_query.base import SelfQueryRetriever
        from langchain.chains.query_constructor.base import (
            StructuredQueryOutputParser,
            get_query_constructor_prompt,
        )

        llm = ChatOllama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=0
        )

        # 定义元数据属性
        metadata_field_info = [
            AttributeInfo(
                name="source",
                description="文档的来源文件名",
                type="string",
            ),
            AttributeInfo(
                name="category",
                description="文档的分类",
                type="string",
            ),
        ]

        document_content_description = "关于人工智能、机器学习等技术的文档"

        # 创建自查询检索器
        self_query_retriever = SelfQueryRetriever.from_llm(
            llm,
            vectorstore,
            document_content_description,
            metadata_field_info,
            verbose=True
        )

        # 测试查询
        query = "找到所有关于technology分类的文档"
        results = self_query_retriever.invoke(query)
        print(f"查询: {query}")
        print(f"自查询检索到 {len(results)} 个结果:")
        for i, doc in enumerate(results):
            print(f"{i + 1}. {doc.page_content[:80]}...")
            print(f"   分类: {doc.metadata.get('category')}")

        return self_query_retriever

    except Exception as e:
        print(f"自查询检索器创建失败: {e}")
        return None


self_query_retriever = self_query_retriever_example()


7. 自查询检索器示例
自查询检索器创建失败: Self query retriever with Vector Store type <class 'langchain_community.vectorstores.faiss.FAISS'> not supported.


## 8. 异步检索示例

In [ ]:


## 8. 异步检索示例

async def async_retrieval_example():
    """异步检索示例"""
    print("\n" + "=" * 60)
    print("8. 异步检索示例")
    print("=" * 60)

    # 8.1 异步单个检索
    print("\n8.1 异步单个检索")
    query = "人工智能的定义"
    result = await vector_retriever.ainvoke(query)
    print(f"异步检索结果数量: {len(result)}")

    # 8.2 异步批量检索
    print("\n8.2 异步批量检索")
    queries = [
        "机器学习算法",
        "深度学习网络",
        "自然语言处理"
    ]

    batch_results = await vector_retriever.abatch(queries)
    print(f"批量检索了 {len(queries)} 个查询")
    for i, results in enumerate(batch_results):
        print(f"查询 {i + 1}: {len(results)} 个结果")


# 运行异步示例
await async_retrieval_example()

In [ ]:

## 9. 自定义检索器

from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from typing import List


class CustomKeywordRetriever(BaseRetriever):
    """自定义关键词检索器"""

    documents: List[Document]
    k: int = 3

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """检索相关文档"""
        query_words = set(query.lower().split())
        scored_docs = []

        for doc in self.documents:
            content_words = set(doc.page_content.lower().split())
            score = len(query_words.intersection(content_words))
            if score > 0:
                scored_docs.append((doc, score))

        # 按分数排序并返回前k个
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, score in scored_docs[:self.k]]


def custom_retriever_example():
    """自定义检索器示例"""
    print("\n" + "=" * 60)
    print("9. 自定义检索器示例")
    print("=" * 60)

    # 创建自定义检索器
    custom_retriever = CustomKeywordRetriever(
        documents=documents,
        k=3
    )

    query = "机器学习人工智能"
    results = custom_retriever.invoke(query)
    print(f"查询: {query}")
    print(f"自定义检索到 {len(results)} 个结果:")
    for i, doc in enumerate(results):
        print(f"{i + 1}. {doc.page_content[:80]}...")


custom_retriever_example()

In [ ]:

## 10. 检索器性能比较

import time


def performance_comparison():
    """检索器性能比较"""
    print("\n" + "=" * 60)
    print("10. 检索器性能比较")
    print("=" * 60)

    query = "深度学习神经网络"
    retrievers = {
        "向量检索": vector_retriever,
        "BM25检索": bm25_retriever,
        "集成检索": ensemble_retriever,
    }

    if multi_query_retriever:
        retrievers["多查询检索"] = multi_query_retriever

    results = {}

    for name, retriever in retrievers.items():
        try:
            start_time = time.time()
            docs = retriever.invoke(query)
            end_time = time.time()

            results[name] = {
                "时间": f"{(end_time - start_time):.3f}秒",
                "结果数": len(docs),
                "首个结果": docs[0].page_content[:50] + "..." if docs else "无结果"
            }
        except Exception as e:
            results[name] = {"错误": str(e)}

    print(f"查询: {query}")
    print("\n性能比较结果:")
    for name, result in results.items():
        print(f"\n{name}:")
        for key, value in result.items():
            print(f"  {key}: {value}")


performance_comparison()


In [8]:

## 11. 实际应用场景

def practical_rag_example():
    """实际RAG应用示例"""
    print("\n" + "=" * 60)
    print("11. 实际RAG应用示例")
    print("=" * 60)

    from langchain.chains import RetrievalQA
    from langchain.prompts import PromptTemplate

    try:
        llm = ChatOllama(
            base_url="http://localhost:11434",
            model="qwen2.5:3b",
            temperature=0.7
        )

        # 自定义提示模板
        prompt_template = """基于以下上下文信息回答问题。如果上下文中没有相关信息，请说"根据提供的信息无法回答"。

上下文:
{context}

问题: {question}

回答:"""

        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

        # 创建RAG链
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=ensemble_retriever,
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )

        # 测试问题
        questions = [
            "什么是机器学习？",
            "深度学习和机器学习有什么区别？",
            "人工智能有哪些应用领域？"
        ]

        for question in questions:
            print(f"\n问题: {question}")
            result = qa_chain.invoke({"query": question})
            print(f"回答: {result['result']}")
            print(f"参考文档数量: {len(result['source_documents'])}")

    except Exception as e:
        print(f"RAG示例创建失败: {e}")


practical_rag_example()

print("\n🎉 所有检索器示例运行完成！")


3. BM25检索器示例

3.1 基础BM25检索
查询: 机器学习算法
BM25检索到 3 个结果:
1. 数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。...
2. Python是一种高级编程语言，广泛用于数据科学、机器学习和Web开发。...
3. 今天天气很好，阳光明媚，适合户外活动和散步。...

3.2 自定义BM25参数
自定义BM25检索结果数量: 2

4. 集成检索器示例

4.1 向量+BM25集成检索
查询: 深度学习神经网络
集成检索到 5 个结果:
1. 数据科学结合了统计学、计算机科学和领域专业知识来从数据中提取洞察。...
2. 深度学习是机器学习的一个分支，使用多层神经网络来模拟人脑的工作方式。...
3. 机器学习是人工智能的一个子集，使计算机能够在没有明确编程的情况下学习和改进。...
4. Python是一种高级编程语言，广泛用于数据科学、机器学习和Web开发。...
5. 今天天气很好，阳光明媚，适合户外活动和散步。...

4.2 多种检索器集成
多重集成检索器创建失败: Could not import scikit-learn, please install with `pip install scikit-learn`.

5. 多查询检索器示例

5.1 基础多查询检索
多查询检索器创建失败: model "qwen2.5:3b" not found, try pulling it first (status code: 404)

6. 上下文压缩检索器示例

6.1 LLM链提取器
上下文压缩检索器创建失败: model "qwen2.5:3b" not found, try pulling it first (status code: 404)

7. 自查询检索器示例
自查询检索器创建失败: Self query retriever with Vector Store type <class 'langchain_community.vectorstores.faiss.FAISS'> not supported.

8. 异步检索示例

8.1 异步单个检索
异步检索结果数量: 3

8.2 异步批量检索
批量检索了 3 个查询
查询 1: 3 个结果
查询 2: 3 个结果
查询

In [9]:
import os

# 设置USER_AGENT环境变量
os.environ["USER_AGENT"] = "LangChain-Tutorial/1.0 (Educational Purpose)"

# 或者设置更详细的USER_AGENT
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

print("✅ USER_AGENT 环境变量已设置")
print(f"当前 USER_AGENT: {os.environ.get('USER_AGENT')}")

✅ USER_AGENT 环境变量已设置
当前 USER_AGENT: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36


In [10]:
"""
LangChain 0.3 Data Connection 完整示例
包含文档加载、文本分割、向量化、存储和检索的完整流程
"""

import os
from typing import List, Dict, Any
import asyncio

# 核心导入
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    WebBaseLoader,
    DirectoryLoader
)
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter
)
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import (
    FAISS,
    Chroma,
    Qdrant
)
from langchain_core.documents import Document
from langchain.retrievers import (
    BM25Retriever,
    EnsembleRetriever,
    MultiQueryRetriever
)
from langchain_ollama import ChatOllama

print("✅ 所有库导入成功")

✅ 所有库导入成功


In [11]:


# 5. Retrievers 示例
def retrievers_example(vectorstore, chunks: List[Document]):
    """检索器示例"""
    print("\n" + "=" * 60)
    print("5. Retrievers 检索器示例")
    print("=" * 60)

    # 5.1 向量存储检索器
    print("\n5.1 向量存储检索器")
    if vectorstore:
        vector_retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )

        results = vector_retriever.invoke("人工智能的应用")
        print(f"向量检索结果数量: {len(results)}")
        for i, doc in enumerate(results):
            print(f"{i + 1}. {doc.page_content[:100]}...")

    # 5.2 BM25检索器
    print("\n5.2 BM25检索器")
    try:
        bm25_retriever = BM25Retriever.from_documents(chunks)
        bm25_retriever.k = 3

        bm25_results = bm25_retriever.invoke("人工智能发展")
        print(f"BM25检索结果数量: {len(bm25_results)}")
        for i, doc in enumerate(bm25_results):
            print(f"{i + 1}. {doc.page_content[:100]}...")

    except Exception as e:
        print(f"BM25检索器创建失败: {e}")
        bm25_retriever = None

    # 5.3 集成检索器
    print("\n5.3 集成检索器")
    if vectorstore and bm25_retriever:
        try:
            ensemble_retriever = EnsembleRetriever(
                retrievers=[vector_retriever, bm25_retriever],
                weights=[0.7, 0.3]  # 向量搜索权重0.7，BM25权重0.3
            )

            ensemble_results = ensemble_retriever.invoke("机器学习技术")
            print(f"集成检索结果数量: {len(ensemble_results)}")

        except Exception as e:
            print(f"集成检索器创建失败: {e}")

    # 5.4 多查询检索器
    print("\n5.4 多查询检索器")
    if vectorstore:
        try:
            llm = ChatOllama(
                base_url="http://localhost:11434",
                model="gemma3:4b"
            )

            multi_query_retriever = MultiQueryRetriever.from_llm(
                retriever=vector_retriever,
                llm=llm
            )

            multi_results = multi_query_retriever.invoke("AI的未来发展趋势")
            print(f"多查询检索结果数量: {len(multi_results)}")

        except Exception as e:
            print(f"多查询检索器创建失败: {e}")
# 5. 检索器
retrievers_example(faiss_store, chunks)

NameError: name 'faiss_store' is not defined